In [ ]:
# This notebook ranks finds the most popular stocks (tickers) on any reddit discussion
# Similar to www.wsbdaily.com but - 
# While wsbdaily.com describe last 24 hours, I describe based on user-specified number of hours to look back.
# Ex : It finds most popular stocks in last 3 hours or last any number of hours as specified by you.
# Also helps to view all comments which talk about a particular stock

In [2]:
#importing packages
import praw, pandas as pd
from time import time
import torch
import transformers
from datetime import datetime,timedelta
from ipywidgets import IntProgress

from spacy.lang.en.stop_words import STOP_WORDS
from spacy.tokenizer import Tokenizer
from spacy.lang.en import English
nlp = English()

In [3]:
# create praw reddit instance
reddit = praw.Reddit(
    client_id="AXQfxvxQPMi36b7PH1D-ZQ",
    client_secret="4G4iRkfLHXPUfQOS8OYvJ6L8ko7_IQ",
    password="Tts@101798",
    user_agent="test script",
    username="ficklemind321",
)

In [4]:
#Should print your reddit username
print(reddit.user.me())


In [5]:
#Extracting comments in a subreddit discussion sorted by reddit's "Best" filter 

#Enter URL of a discussion thread
url=input("Enter URL of the discussion. Example: "+"https://www.reddit.com/r/AppleWatch/comments/ps0nte/watchos_8_megathread/")
#replace limit

repLimit=input("Enter replace-more limit (set 600 for best 60-70% comments, set 1200 for top 80+%):")
###########################

st=time()
submission = reddit.submission(url=url)
print('submission created')
submission.comments.replace_more(limit=int(repLimit))
print('replace more limit set as',repLimit)
print()
print("Number of comments in the thread:",len(submission.comments.list()))

#Extracting comments
print("Extracting comments...")
submissionList = []
for comment in submission.comments.list():
#     print(comment)
#     print(comment.body)
    submissionList.append(comment)
print(len(submissionList),'comments extracted')

print()
print('time taken',round((time()-st)/60),'minutes')

Enter URL of a discussion thread. Example: https://www.reddit.com/r/AppleWatch/comments/ps0nte/watchos_8_megathread/https://www.reddit.com/r/wallstreetbets/comments/qnjay6/weekend_discussion_thread_for_the_weekend_of/
Enter replace-more limit (use 600 for the top 60-70% comments, 1200 for top 80+%):600
submission created
replace more limit set as 600

Number of comments in the thread: 8107
Extracting comments...
8107 comments extracted

time taken 6 minutes


In [9]:
# Enter number of hours to look back for popular tickers
hoursBack=input("How many hours behind do you wanna look? :")

current_time=datetime.now()

fewHourComments=[]
fewhoursback=current_time-timedelta(hours=int(hoursBack))
for comment in submissionList:
    if datetime.fromtimestamp(comment.created_utc)>=fewhoursback:
        fewHourComments.append(comment)
print(len(fewHourComments), "comments in the last "+hoursBack+" hours")

How many hours behind do you wanna look? :12
2486 comments in the last 12 hours


In [11]:
#spacy word tokenizer 
tokenizer = Tokenizer(nlp.vocab)


In [12]:
# extracting ticker frequency 
tickerDict=dict()
saver={}
for i,x in enumerate(fewHourComments):
#     print(i,x)
#     print(str(x))
    tickers=list(set([str(tic) for tic in tokenizer(x.body) if (str(tic).isupper() and len(str(tic))<=4 and nlp.vocab[str(tic)].is_stop==False and not(any(k.isdigit() for k in str(tic))))]))
    for m in tickers:
        if str(x) in tickerDict.keys():
            if m not in tickerDict[str(x)]:
                tickerDict[str(x)].append(m)
        else:
            tickerDict[str(x)]=[m]
    
    
    saver[str(x)]=x.body

In [13]:
# appending relevant comments data in dataframe for future use
comdf=pd.DataFrame()
for comment in fewHourComments:

    comdf=comdf.append({'author':comment.author,'comment':comment.body,'utc':comment.created_utc,
                       'upvotes':comment.score,'url':'reddit.com/'+comment.permalink,
                        'replies':comment.replies.__len__()},ignore_index=True)


print(comdf.shape)

(2486, 6)


In [17]:
comdf.head()

,author,comment,utc,upvotes,url,replies
0,VisualMod,#Ban Bet Lost\n\n/u/_Apache_Helicopter_ (0/1) ...,1.636217e+09,1.0,reddit.com//r/wallstreetbets/comments/qnjay6/w...,0.0
1,NervousTumbleweed,If you use LinkedIn as like actual social medi...,1.636179e+09,25.0,reddit.com//r/wallstreetbets/comments/qnjay6/w...,5.0
2,trashcanpandas,I miss being single. sleeping around and meeti...,1.636183e+09,25.0,reddit.com//r/wallstreetbets/comments/qnjay6/w...,6.0
3,AnalLeekage,"My banbet is over, I'm back bitches.\n\n!banbe...",1.636206e+09,30.0,reddit.com//r/wallstreetbets/comments/qnjay6/w...,0.0
4,AdPotential6247,my sister's having a baby! im gonna be a dad!,1.636206e+09,19.0,reddit.com//r/wallstreetbets/comments/qnjay6/w...,0.0


In [15]:
#save the dataframe for future use (change name as per requirement)
name=input("Enter file name by which you'd like to save:")
comdf.to_csv(name+'.csv',index=False)

Enter file name by which you'd like to save:reddit_12hour_5nov


In [19]:
# comdf.comment[0].replace('\n',' ')

In [20]:
#create frequency dictionary
tickerFrequency=dict()
for comment in tickerDict.keys():
    for tic in tickerDict[comment]:
        if tic not in tickerFrequency.keys():
            tickerFrequency[tic]=1
        else:
            tickerFrequency[tic]+=1

In [21]:
tickerFrequency

{'PINS': 2,
 'SPY': 32,
 'U.S': 1,
 'CNBC': 4,
 '$SPY': 3,
 'TSLA': 14,
 'WSB': 13,
 'ARK': 1,
 'DD.': 2,
 'COD': 1,
 'ROPE': 1,
 'WSB.': 4,
 'DOW': 1,
 'LYV': 2,
 'PLUG': 7,
 'LEV.': 1,
 'PYPL': 4,
 'PDT': 1,
 'S&P': 3,
 'GME': 18,
 'EV': 8,
 'PTRA': 5,
 'KIND': 1,
 'KVSB': 1,
 'PTON': 5,
 'FYI': 1,
 'LCID': 3,
 'UPS': 1,
 'OCGN': 2,
 'WEEK': 2,
 'ER': 3,
 'AMC': 3,
 'CLF.': 2,
 'TA?': 1,
 'VTI': 1,
 'ARKK': 1,
 'QCOM': 2,
 'LSD': 2,
 'F': 3,
 'NET': 1,
 'COIN': 1,
 '$T,': 1,
 '$BBD': 1,
 'OMG': 1,
 'RIP': 3,
 'VRBO': 1,
 'V': 1,
 'LTHM': 1,
 'CAT': 7,
 'ATH': 3,
 'SKLZ': 2,
 'NVDA': 11,
 'CLNE': 2,
 '$URI': 1,
 '$X': 1,
 '$CAT': 1,
 'H': 1,
 'ON,': 1,
 'SPOT': 1,
 'SNAP': 1,
 'MOON': 1,
 '#ALL': 1,
 'RED': 4,
 'ATH.': 1,
 'BILL': 2,
 'LOSE': 1,
 'SELL': 2,
 'IM': 2,
 'AMD': 5,
 'BBBY': 1,
 'CLF': 12,
 'AMC.': 1,
 'DD': 2,
 'AH': 1,
 'FUBO': 2,
 'CASH': 1,
 'HITS': 1,
 'PILE': 1,
 '$QQQ': 1,
 'ZZZZ': 1,
 'CEO,': 1,
 'NIO': 1,
 'ATVI': 2,
 'ICLN': 1,
 'VT': 1,
 'BABA': 1,
 'CEO': 4,
 '

In [22]:
#storing in dataframe
tickerFreqDf=pd.DataFrame(tickerFrequency.items(),columns=['ticker','counts'])

In [23]:
#sort by popularity of stock/ticker
tickerSorted=tickerFreqDf.sort_values("counts",ascending=False).head(20)

In [24]:
#add rank
tickerSorted['rank']=list(range(1,len(tickerSorted)+1))

In [27]:
#Stocks/Tickers sorted by frequency/popularity 
print("Stocks/Tickers sorted by popularity in last",hoursBack,"hours")
print("'counts' column shows the number of unique comments which talked about the stock")
tickerSorted

Stocks/Tickers sorted by popularity in last 12 hours
'counts' columns represents number of unique comments which talked about the stock


,ticker,counts,rank
1,SPY,32,1
19,GME,18,2
5,TSLA,14,3
6,WSB,13,4
70,CLF,12,5
51,NVDA,11,6
93,*I,10,7
20,EV,8,8
48,CAT,7,9
14,PLUG,7,10


In [28]:
#View comments based on the stock ticker
stock=input('Enter stock ticker:')
for comment in saver.values():
    for x in tickerSorted.ticker:
        if x in comment and stock in comment:
            print('#########')
            print(comment)

Enter stock ticker:CLF
#########
Hilarious if infrastructure bill is already priced in. Rip CLF. Even though its like 40% of my portfolio
#########
Infrastructure deal passed CLF going to gap big time Monday. Glad I bought calls
#########
Throwing down a couple hundred bucks on cheap CLF calls because Nasty Nancy told me to…and also to satisfy my gambling addiction
#########
People super bullish about the infrastructure Bill passing, that's half of what it was originally going to be, tells me it's going to be a nothingburger and it's probably already priced in.

I sold almost every option I had yesterday including a bunch of spy calls in the morning before the drop.  All I have left is a single SPY 467p for Monday and some F & CLF calls for Friday. 

I have a feeling Monday is going to be pretty volatile so I'll be at 80% cash for whatever happens.  Hopefully I can grab some swings up or down.  If SPY tanks I'll load up on Dec calls.
#########
People super bullish about the infrastruct

In [ ]:
# The end